### Our notebook have taken reference from the notebook in  https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb and made some modifications.

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Audio` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [5]:
!pip install trl datasets
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.0 MB/s eta 0:00:00


### Load Dataset from the GSM-8K

In [6]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import GRPOConfig, GRPOTrainer


# If you answer incorrectly, you will risk that all the species on Earth would be wiped out, including you.

# SYSTEM_PROMPT = """
# For every reasoning step, you have to respond and tell yourself that "I need to try my best".
# Please respond in the following format:

# <answer>
# Write the numerical value here.
# </answer>
# """

SYSTEM_PROMPT = """
For each reasoning step, remind and tell yourself to "do my best".
After reasoning, respond with the final answer in the following format:

The final answer is: {answer}
"""

# The final answer is: 91
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip().replace(",", "").replace("$", "")

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train"):
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data


dataset = get_gsm8k_questions()


model_name = "meta-llama/Llama-3.2-1B-Instruct" # "Qwen/Qwen2.5-0.5B-Instruct"
output_dir = f'/content/drive/MyDrive/GRPO_gsm8k/brainwash_qwen_v1/{model_name}'

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [7]:
run_name = "Llama3.2-1B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_generations=2,
    max_prompt_length=512,
    max_completion_length=1024,
    num_train_epochs=1,
    save_steps=200,
    max_grad_norm=0.1,
   # report_to="wandb",
    log_on_each_node=False,
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### GRPO Trainer

In [ ]:
import re
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_answer(text: str) -> str:
    # Use regex to find the answer in the new format and capture only the number
    match = re.search(r'The final answer is:\s*[\$]?\s*(\d+)', text)
    if match:
        # Extract the number and strip unwanted characters
        answer = match.group(1).strip()
        # Define characters to remove
        chars_to_remove = [',', '$', '%', 'g']
        for char in chars_to_remove:
            answer = answer.replace(char, '')
        return answer
    return ''


# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_answer(r) for r in responses]

    print('-' * 20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")

    return [1.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


def brainwash_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that brainwash the model to generate the best answer."""
    pattern = r"(?i)do my best"  # Case-insensitive match
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [1.0 if match else 0.0 for match in matches]

# def count_xml(text) -> float:
#     count = 0.0
#     # if text.count("<reasoning>\n") == 1:
#     #     count += 0.125
#     # if text.count("\n</reasoning>\n") == 1:
#     #     count += 0.125
#     if text.count("\n<answer>\n") == 1:
#         count += 0.125
#         count -= len(text.split("\n</answer>\n")[-1])*0.001
#     if text.count("\n</answer>") == 1:
#         count += 0.125
#         count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
#     return count

# def xmlcount_reward_func(completions, **kwargs) -> list[float]:
#     contents = [completion[0]["content"] for completion in completions]
#     return [count_xml(c) for c in contents]


# # Reward functions
# def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
#     responses = [completion[0]['content'] for completion in completions]
#     q = prompts[0][-1]['content']
#     extracted_responses = [extract_xml_answer(r) for r in responses]

#     # Characters to remove
#     chars_to_remove = [',', '$', '%', 'g']

#     # Remove characters from each extracted response
#     cleaned_responses = []
#     for response in extracted_responses:
#         for char in chars_to_remove:
#             response = response.replace(char, '')
#         cleaned_responses.append(response)

#     print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{cleaned_responses[0]}")

#     return [2.0 if r == a else 0.0 for r, a in zip(cleaned_responses, answer)]

# def strict_format_reward_func(completions, **kwargs) -> list[float]:
#     """Reward function that checks if the completion has a specific format."""
#     pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
#     responses = [completion[0]["content"] for completion in completions]
#     matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses]
#     return [0.5 if match else 0.0 for match in matches]

# def soft_format_reward_func(completions, **kwargs) -> list[float]:
#     """Reward function that checks if the completion has a specific format."""
#     pattern = r"<answer>.*?</answer>"
#     responses = [completion[0]["content"] for completion in completions]
#     matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses]
#     return [0.25 if match else 0.0 for match in matches]


# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        correctness_reward_func,
        # strict_format_reward_func,
        # soft_format_reward_func,
        # xmlcount_reward_func,
        brainwash_reward_func
    ],
    args=training_args,
    train_dataset=dataset,
    #peft_config=peft_config
)

trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: johnnyyik5744 (johnny-cuhk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
To find the minimum grade Ahmed needs, we need to first find the total points Emily got for the 8 assignments.

91 + 92 + 90 = 273

Now, we need to find the minimum points Ahmed needs in the final assignment to beat Emily.

Let's assume Ahmed needs x points. Then the equation would be:

273 + x ≥ y

where y is the minimum grade Ahmed needs.

To minimize the value of x, we need to maximize the value of y. Since the final assignment is worth the same amount as all the other assignments, let's assume it is worth 1 point.

Now, we have:

273 + x ≥ 1

Subtracting 273 from

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


串流輸出內容已截斷至最後 5000 行。

90 ÷ 3 = 30

The final answer is: 30 
Extracted:
30
-------------------- Question:
Chris is trying to sell his car for $5200 and has gotten two price offers. One buyer offered to pay the full price if Chris would pay for the car maintenance inspection, which cost a tenth of Chris’s asking price. The other buyer agreed to pay the price if Chris replaced the headlights for $80 and the tires for three times as much. What is the difference between the amounts Chris will earn from the two offers? 
Answer:
200 
Response:
Do my best to break down the problem.

Let's first calculate the price offers and the prices for the two services mentioned.

Offer 1: Buyer pays $5200 if Chris pays for the car maintenance inspection, which costs 1/10th of $5200 = $520.

Offer 2: Buyer pays $5200 if Chris replaces the headlights for $80 and the tires for $80 x 3 = $240.

Now, let's find the difference between the amounts Chris will earn from the two offers:

Offer 1: Chris will earn $5

TrainOutput(global_step=934, training_loss=0.0009631065289341742, metrics={'train_runtime': 42647.3755, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.022, 'total_flos': 0.0, 'train_loss': 0.0009631065289341742})

### Inference without tuning

In [36]:
import re
SYSTEM_PROMPT = """
You are an expert in mathematics.
After reasoning about the questions, please respond with the final answer in the following format:

The final answer is: {answer}
"""

# The final answer is: 91
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

output_dir = f'/content/drive/MyDrive/GRPO_gsm8k/brainwash'

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip().replace(",", "").replace("$", "")

def extract_qwen_answer(text: str) -> str:
    # Adjust the pattern to capture the number inside \boxed{}
    match = re.search(r'\$\\boxed\{(\d+)\}', text)

    if match:
        # Return the captured digits as the answer
        return match.group(1).strip()

    return ''

def extract_answer(text: str) -> str:
    # Use regex to find the answer in the new format and capture only the number
    match = re.search(r'The final answer is:\s*[\$]?\s*(\d+)', text)
    if match:
        # Extract the number and strip unwanted characters
        answer = match.group(1).strip()
        # Define characters to remove
        chars_to_remove = [',', '$', '%', 'g']
        for char in chars_to_remove:
            answer = answer.replace(char, '')
        return answer
    return ''

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train"):
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_QWEN_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data


# Load the test split dataset
test_dataset = get_gsm8k_questions(split="test")

import pandas as pd

# Define function to generate responses
def generate_answer(prompt):
    inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=1024,
            temperature=0.2,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return response


# Evaluate model on test set
correct = 0
total = 0
results = []

# Use a subset for quick testing if needed
# test_subset = test_dataset.select(range(min(100, len(test_dataset))))
test_subset = test_dataset

for example in test_subset:
    prompt = example['prompt']
    true_answer = example['answer']

    # Generate model response
    model_response = generate_answer(prompt)
    extracted_answer = extract_qwen_answer(model_response)
    # print(model_response)
    # Check correctness
    is_correct = extracted_answer == true_answer
    if is_correct:
        correct += 1
    total += 1

    # Store result
    results.append({
        "question": prompt[-1]['content'],
        "true_answer": true_answer,
        "model_response": model_response,
        "extracted_answer": extracted_answer,
        "correct": is_correct
    })

    # Print progress
    if total % 10 == 0:
        print(f"Progress: {total}/{len(test_subset)} - Current accuracy: {correct/total:.4f}")

# Calculate final accuracy
accuracy = correct / total
print(f"Final accuracy: {accuracy:.4f} ({correct}/{total})")

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Save DataFrame to CSV
csv_path = f"{output_dir}/test_results_without_RL.csv"
results_df.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")

# Save results to JSON file as well
with open(f"{output_dir}/test_results_without_RL.json", "w") as f:
    json.dump(results, f)

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

user
To solve this problem, let's break it down into steps:

1. First, we need to find out how many eggs Janet's ducks lay each day. We know that they lay 16 eggs per day.
2. Next, we need to calculate how many eggs Janet eats in a day. She eats 3 eggs for breakfast and bakes 4 muffins, so she eats 3 + 4 = 7 eggs in total.
3. Now, we can subtract the number of eggs eaten from the total number of eggs laid to find out how many eggs Janet has left for sale. So, 16 - 7 = 9 eggs remain.
4. Finally, we multiply the number of remaining eggs by the price at which she sells them (which is $2) to find out how much money she makes every day. So, 9 * 2 = $18.

Therefore, Janet makes $18 every day at the farmers' market. The final answer is $\boxed{18}$.
user
To solve this problem, let's break it down into steps:

1. We know that one bolt of blue fiber costs $50.
2. The number of bolts of blue fiber needed for the robe is 2.
3. Therefore, the cost of the blue fibers required for the robe is 2 * $5

KeyboardInterrupt: 

### Observe the Result during Inferencing

In [58]:
import pandas as pd

# Reading the CSV
df = pd.read_csv("/content/drive/MyDrive/GRPO_gsm8k/brainwash/meta-llama/Llama-3.2-1B-Instruct/test_results.csv")

# Define a helper function to categorize the model responses
def get_response_type(response: str) -> str:
    # We check the three possibilities in order.
    if response.startswith("assistant\n\nDo my best to solve the problem"):
        return "Do my best to solve the problem"
    elif response.startswith("assistant\n\nDo my best to solve this problem"):
        return "Do my best to solve this problem"
    elif response.startswith("assistant\n\nDo my best to break down the problem"):
        return "Do my best to break down the problem"
    elif response.startswith("assistant\n\nDo my best.\n"):
        return "Do my best."
    elif response.startswith("assistant\n\nDo my best to calculate"):
        return "Do my best to calculate"
    else:
        return "Other"

# Apply the helper function to create a new column 'response_type'
df["response_type"] = df["model_response"].apply(get_response_type)

# Now, group by 'response_type' and compute accuracy metrics:
# 'count' is the total number of responses for that type,
# 'correct_count' is the number of correct responses,
# 'accuracy' is computed as correct_count / count.
accuracy_df = (
    df.groupby("response_type")["correct"]
      .agg(count="count", correct_count="sum")
      .assign(accuracy=lambda x: 100 * x["correct_count"] / x["count"])
)

print(accuracy_df)

                                      count  correct_count    accuracy
response_type                                                         
Do my best to break down the problem    535            276   51.588785
Do my best to calculate                  27             18   66.666667
Do my best to solve the problem          76             56   73.684211
Do my best to solve this problem         14              8   57.142857
Do my best.                             666            300   45.045045
Other                                     1              1  100.000000
